### Parte 5: Normalización 

Como bien dice el nombre del fichero, lo que se busca es normalizar cada una de las variables que van a conformar el ```input``` de nuestra red. Para ello, utilizaremos una normalización min-max del 0 al 1 y guardaremos sus factores de escala para una futura desnormalización.

#### 5.1. Normalización conjunto de entrenamiento:

In [1]:
import tensorflow as tf 
import numpy as np 
import warnings
warnings.filterwarnings("ignore")


path_tr= "../../parches_winscp/bicubic_32_32/190824_16_tr/raw/"

var_names = ["u10_input", "u10_target","v10_input", "v10_target",
             "t2_input", "th2_input", "psfc_input", "pblh_input", 
             "xland_input", "hgt_input", "aspect_input", "slope_input",
             "EmaxU_input", "EmaxV_input", "EminU_input", "EminV_input"] 


In [2]:
carpeta_tr= '../../parches_winscp/bicubic_32_32/190824_16_tr/norm/'
factores_escala = {} 

for name in var_names: 
              
    array_tr = tf.convert_to_tensor(np.load(f"{path_tr}{name}_raw_190824_16_tr.npy"))
    factor_min = tf.reduce_min(array_tr)
    factor_max = tf.reduce_max(array_tr)

    array_norm_tr = (array_tr - factor_min) / (factor_max - factor_min)
    np.save(carpeta_tr+name+"_norm_190824_16_tr.npy", array_norm_tr)
    
    factores_escala[name] = [factor_min.numpy(), factor_max.numpy() ]

factores_escala

{'u10_input': [-24.035671, 19.812311],
 'u10_target': [-31.29994, 31.842432],
 'v10_input': [-20.810207, 14.965427],
 'v10_target': [-30.851156, 23.74102],
 't2_input': [257.75668, 298.89575],
 'th2_input': [267.9052, 303.7989],
 'psfc_input': [82636.02, 103507.99],
 'pblh_input': [-273.23758, 2129.6934],
 'xland_input': [0.761478, 2.2074287],
 'hgt_input': [-22.215477, 1449.4507],
 'aspect_input': [0.0, 360.0],
 'slope_input': [0.0, 89.29281],
 'EmaxU_input': [-23.877573, 17.691437],
 'EmaxV_input': [-19.628057, 13.7348995],
 'EminU_input': [-17.493675, 23.955181],
 'EminV_input': [-13.671599, 19.655247]}

In [3]:
import pickle

with open('extra/factores/factores_escala_190824_16.pkl', 'wb') as archivo:
    pickle.dump(factores_escala, archivo)

#### 5.2. Normalización conjunto de validación usando factores de escala anteriores.

In [3]:
path_va   = "../../parches_winscp/bicubic_32_32/190824_16_va/raw/"
carpeta_va= '../../parches_winscp/bicubic_32_32/190824_16_va/norm/'

for name in var_names: 
    array_va = tf.convert_to_tensor(np.load(f"{path_va}{name}_raw_190824_16_va.npy"))
    array_norm_va = (array_va - factores_escala[name][0]) / (factores_escala[name][1] - factores_escala[name][0])
    np.save(carpeta_va+name+"_norm_190824_16_va.npy", array_norm_va)

#### 5.3. Chequeo desnormalización
Tanto para el conjunto de entrenamiento como de validación se observan diferencias asociadas al punto flotante

In [4]:
for var in var_names:

    array_norm   = tf.convert_to_tensor(np.load(f"{carpeta_tr}{var}_norm_190824_16_tr.npy"))
    array_desnorm =( array_norm * (factores_escala[var][1]- factores_escala[var][0])  ) + factores_escala[var][0]
    array_original = tf.convert_to_tensor(np.load(f"{path_tr}{var}_raw_190824_16_tr.npy"))

    print(f"Dif. máx. array og versus desnorm {var}", np.max( np.abs( array_desnorm - array_original)) )


Dif. máx. array og versus desnorm u10_input 2.861023e-06
Dif. máx. array og versus desnorm u10_target 2.861023e-06
Dif. máx. array og versus desnorm v10_input 2.861023e-06
Dif. máx. array og versus desnorm v10_target 3.8146973e-06
Dif. máx. array og versus desnorm t2_input 0.0
Dif. máx. array og versus desnorm th2_input 0.0
Dif. máx. array og versus desnorm psfc_input 0.0
Dif. máx. array og versus desnorm pblh_input 0.00018310547
Dif. máx. array og versus desnorm xland_input 1.1920929e-07
Dif. máx. array og versus desnorm hgt_input 6.1035156e-05
Dif. máx. array og versus desnorm aspect_input 1.5258789e-05
Dif. máx. array og versus desnorm slope_input 3.8146973e-06
Dif. máx. array og versus desnorm EmaxU_input 2.861023e-06
Dif. máx. array og versus desnorm EmaxV_input 2.861023e-06
Dif. máx. array og versus desnorm EminU_input 2.861023e-06
Dif. máx. array og versus desnorm EminV_input 3.8146973e-06


Para el conjunto de validación: 

In [6]:
for var in var_names:

    array_norm   = tf.convert_to_tensor(np.load(f"{carpeta_va}{var}_norm_190824_16_va.npy"))
    array_desnorm =( array_norm * (factores_escala[var][1]- factores_escala[var][0])  ) + factores_escala[var][0]
    array_original = tf.convert_to_tensor(np.load(f"{path_va}{var}_raw_190824_16_va.npy"))

    print(f"Dif. máx. array og versus desnorm {var}", np.max( np.abs( array_desnorm - array_original)) )

Dif. máx. array og versus desnorm u10_input 2.861023e-06
Dif. máx. array og versus desnorm u10_target 7.6293945e-06
Dif. máx. array og versus desnorm v10_input 2.861023e-06
Dif. máx. array og versus desnorm v10_target 3.8146973e-06
Dif. máx. array og versus desnorm t2_input 0.0
Dif. máx. array og versus desnorm th2_input 0.0
Dif. máx. array og versus desnorm psfc_input 0.0
Dif. máx. array og versus desnorm pblh_input 0.00024414062
Dif. máx. array og versus desnorm xland_input 1.1920929e-07
Dif. máx. array og versus desnorm hgt_input 6.1035156e-05
Dif. máx. array og versus desnorm aspect_input 1.5258789e-05
Dif. máx. array og versus desnorm slope_input 3.8146973e-06
Dif. máx. array og versus desnorm EmaxU_input 5.722046e-06
Dif. máx. array og versus desnorm EmaxV_input 2.861023e-06
Dif. máx. array og versus desnorm EminU_input 2.861023e-06
Dif. máx. array og versus desnorm EminV_input 5.722046e-06
